## Filter MERSCOPE unsuitable genes

Want to create lists of genes that shouldn't be used for gene panel selection, due to either proven track record of poor performance or unsuitable length

In [1]:
import pickle

In [12]:
# These genes are from an email from Brian Long labelling them as "the worst offender" 
# [the "perhaps less bad" genes are not included here]
previousGenes = ['Cdh7', 'Col25a1', 'Foxp2', 'Gabra1', 'Gabrg1', 'Gad1', 'Inpp4b', 'Lypd1', 'Maob', 
            'Nell1', 'Pantr1', 'Pkib', 'Shisa9', 'Sox5', 'Thsd7a', 'Tmem196',  'Tunar']

with open("../Data/previousGenes.pickle", "wb") as f:
    pickle.dump(previousGenes,f)

## Gene length via ensembl .gtf genome

Search individual genes: https://www.ncbi.nlm.nih.gov/gene/21419

In [1]:
import numpy as np
import pandas as pd
import anndata as ad
import scanpy as sc
import json
import pickle
import numpy as np
from gtfparse import read_gtf

In [2]:
# Load in pre-processed data from glutamatergic class designation, subclass to all rank_gene_groups already performed
gluData = sc.read("../Data/devData.h5ad")

C:\Users\mathew.summers\Anaconda3\envs\scanpy1.9\lib\site-packages\anndata\_core\anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)


In [7]:
# Get gene names from scRNAseq data set
seqGenes = list(gluData.var_names)

# Load genome file as dataframe
mouseGTF = read_gtf("../Data/Mouse Genome/Mus_musculus.GRCm39.106.gtf")

# Subset dataframe to entries labeled as genes [df.feature.unique() to view possible entries, e.g. transcript]
mouseGenes = mouseGTF[mouseGTF.feature == "gene"]

C:\Users\mathew.summers\Anaconda3\envs\scanpy1.9\lib\site-packages\gtfparse\read_gtf.py:82: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  chunk_iterator = pd.read_csv(
C:\Users\mathew.summers\Anaconda3\envs\scanpy1.9\lib\site-packages\gtfparse\read_gtf.py:82: FutureWarning: The warn_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  chunk_iterator = pd.read_csv(
INFO:root:Extracted GTF attributes: ['gene_id', 'gene_version', 'gene_name', 'gene_source', 'gene_biotype', 'transcript_id', 'transcript_version', 'transcript_name', 'transcript_source', 'transcript_biotype', 'tag', 'transcript_support_level', 'exon_number', 'exon_id', 'exon_version', 'ccds_id', 'protein_id', 'protein_version']


In [6]:
# Do a quick and dirty count of how many seqGene names don't match a gene within the database
a = 0
for gene in seqGenes:
    a += not(gene in list(mouseGenes.gene_name))

print(a) # Should be ~750 out of ~40,000 in development data set, not bad

729


In [8]:
# Get length of each genome entry
mouseGenes["length"] = mouseGenes.end - mouseGenes.start

C:\Users\mathew.summers\AppData\Local\Temp\ipykernel_14616\946826167.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  mouseGenes["length"] = mouseGenes.end - mouseGenes.start


In [39]:
# Want to find which genes within the list of seqGenes is less than 3kb long

shortGenes = list();

for gene in seqGenes:
    # First check for genes matching sequencing gene name
    geneDF = mouseGenes[mouseGenes.gene_name == gene]
    if len(geneDF) > 0:
        # Then check if such genes are shorter than 3kb 
        # [some genes have multiple listings, just be conservative and use largest]
        if np.max(geneDF.length < 3000):
            shortGenes.append(gene)
    
print('Found ', len(shortGenes), ' short genes.')

Found  5696  short genes.


In [40]:
with open("../Data/shortGenes.pickle", "wb") as f:
    pickle.dump(shortGenes,f)

In [47]:
list(set(seqGenes) - set(shortGenes))

['Gm29622',
 '2810408A11Rik',
 'Fgf18',
 'Gm26811',
 'Dnajb9',
 'Crygs',
 'Aicda',
 'C87487',
 'Mamld1',
 'Ifi208',
 'Gm28672',
 'Gmps',
 'Cyp2g1',
 'Macrod1',
 'Ttll3',
 'Papss1',
 'Hist1h3a',
 'Pon1',
 'Btnl10',
 'Rpl18',
 'Gm26685',
 '4930434J06Rik',
 'Gpc2',
 'Mfsd7a',
 'Pigq',
 'Dnajc17',
 'Lysmd4',
 'Gm16029',
 'Gm4724',
 'Gm40916',
 'Rs1',
 'Vmn2r72',
 'Gm15608',
 'Cfap410',
 'Nkpd1',
 'Bok',
 'Hnf4g',
 'Gm42817',
 'Snrnp200',
 'Rnf225',
 'Usp43',
 'B230217J21Rik',
 'Gm16253',
 'Olfr193',
 'Cycs',
 'Gm44127',
 'Emsy',
 'Ptk2',
 'Gm3727',
 'Gm33195',
 'Gm31831',
 'Vcpip1',
 'C230099D08Rik',
 'Zfp703',
 'Pi15',
 'Bcl2l12',
 'Camk2g',
 'Rab11fip2',
 'Gm36043',
 'Wdr12',
 'Gm31182',
 'Crebzf',
 'Zfp36l1',
 'Msrb2',
 'Cerkl',
 'Kif21a',
 'Gm15346',
 'Gm50350',
 'AY761184',
 'Vmn2r1',
 'Cabcoco1',
 'Fastkd2',
 'Gm6710',
 'Psd3',
 'Pbx3',
 'Eapp',
 '1810021B22Rik',
 'Lgi2',
 'Hist1h3g',
 'Gusb',
 'Rnf126',
 'Xist',
 'Sim2',
 '5430416N02Rik',
 'Elmod1',
 'Fabp12',
 'Phf20l1',
 'Acad9',


## Save both previous and short genes on a collated list

In [16]:
import json

badGenes = list(np.unique([previousGenes + shortGenes]))

with open("../Data/badGenes.json", "w") as f:
    json.dump(badGenes, f, indent=4)

## Do inverse, get a list of probable good genes

In [6]:
# Don't want to select from unsuitable genes, so load list of genes that are suspected to be unsuitable for MERSCOPE
with open("../Data/badGenes.json", "r") as f:
    badGenes = json.load(f)
    
# Find remaining genes after exclusion
keepGenes = list(set(gluData.var_names) - set(badGenes))

with open("../Data/keepGenes.json", "w") as f:
    json.dump(keepGenes, f, indent=4)

## Misc Analysis

In [74]:
x.iloc[50000:50010,[0, 3, 4, 10]]

,seqname,start,end,gene_name
669158,6,90666713,90667811,Gm44123
669162,6,90770878,90771457,Gm44105
669166,6,90821963,90822442,Rpl21-ps12
669169,6,90834519,90834614,Gm25185
669172,6,90882801,90883435,Ppp1r2-ps2
669175,6,124835407,124840880,Gpr162
669190,6,124835410,124840675,Gpr162
669195,6,124835411,124840946,Gpr162
669212,6,124838694,124840675,Gpr162
669217,6,145061379,145120654,Irag2


In [94]:
z = x[x.gene_name == 'Nrxn1']
z.end - z.start

1654644    1055276
1654691    1059427
1654742      52980
1654753    1053497
1654800     420072
1654817      52130
1654828     419589
1654847    1052057
1654895      51287
1654908    1051846
1654954     419054
1654973    1055837
1655020     171098
1655029     187055
1655034     246534
1655038     884515
1655057     226140
1655068     226048
1655079       2796
1655081     553277
1655095      40308
1655100     110580
1655107       9979
1655110     392840
1655115     389615
1655119       7295
dtype: int64

In [95]:
z.iloc[:,8:]

,gene_id,gene_version,gene_name,gene_source,gene_biotype,transcript_id,transcript_version,transcript_name,transcript_source,transcript_biotype,tag,transcript_support_level,exon_number,exon_id,exon_version,ccds_id,protein_id,protein_version
1654644,ENSMUSG00000024109,19,Nrxn1,ensembl_havana,protein_coding,ENSMUST00000072671,14,Nrxn1-202,havana,protein_coding,basic,5,,,,,,
1654691,ENSMUSG00000024109,19,Nrxn1,ensembl_havana,protein_coding,ENSMUST00000160844,10,Nrxn1-207,havana,protein_coding,"CCDS,basic",1,,,,CCDS57113,,
1654742,ENSMUSG00000024109,19,Nrxn1,ensembl_havana,protein_coding,ENSMUST00000197268,5,Nrxn1-225,havana,protein_coding,"CCDS,basic",3,,,,CCDS84339,,
1654753,ENSMUSG00000024109,19,Nrxn1,ensembl_havana,protein_coding,ENSMUST00000174331,8,Nrxn1-216,havana,protein_coding,basic,5,,,,,,
1654800,ENSMUSG00000024109,19,Nrxn1,ensembl_havana,protein_coding,ENSMUST00000159778,8,Nrxn1-204,havana,protein_coding,"CCDS,basic",5,,,,CCDS84340,,
1654817,ENSMUSG00000024109,19,Nrxn1,ensembl_havana,protein_coding,ENSMUST00000173917,8,Nrxn1-215,havana,protein_coding,"CCDS,basic",2,,,,CCDS84338,,
1654828,ENSMUSG00000024109,19,Nrxn1,ensembl_havana,protein_coding,ENSMUST00000174337,8,Nrxn1-217,ensembl_havana,protein_coding,"CCDS,basic",5,,,,CCDS84341,,
1654847,ENSMUSG00000024109,19,Nrxn1,ensembl_havana,protein_coding,ENSMUST00000161402,10,Nrxn1-209,havana,protein_coding,basic,5,,,,,,
1654895,ENSMUSG00000024109,19,Nrxn1,ensembl_havana,protein_coding,ENSMUST00000173222,2,Nrxn1-214,havana,protein_coding,basic,1,,,,,,
1654908,ENSMUSG00000024109,19,Nrxn1,ensembl_havana,protein_coding,ENSMUST00000054059,15,Nrxn1-201,havana,protein_coding,basic,5,,,,,,


In [45]:
z.iloc[:,8:]

,gene_id,gene_version,gene_name,gene_source,gene_biotype,transcript_id,transcript_version,transcript_name,transcript_source,transcript_biotype,tag,transcript_support_level,exon_number,exon_id,exon_version,ccds_id,protein_id,protein_version
108722,ENSMUSG00000036907,12,C1ql2,ensembl_havana,protein_coding,ENSMUST00000037286,12,C1ql2-201,ensembl_havana,protein_coding,"CCDS,basic",1,,,,CCDS15233,,


In [69]:
geneDF = df[df.feature == "gene"]
geneDF["length"] = geneDF.end - geneDF.start
print(len(geneDF))
print(sum(geneDF["length"] < 3000))

55414
26736


C:\Users\mathew.summers\AppData\Local\Temp\ipykernel_13536\4003160007.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  geneDF["length"] = geneDF.end - geneDF.start
